# Lab | Web Scraping

¡Bienvenido al laboratorio de aventura de web scraping "Books to Scrape"!

**Objetivo**

En este laboratorio, nos embarcaremos en una misión para descubrir valiosos conocimientos a partir de los datos disponibles en Books to Scrape, una plataforma en línea que muestra una amplia variedad de libros. Como analista de datos, se te ha encomendado la tarea de extraer un subconjunto específico de datos de libros de Books to Scrape para ayudar a las editoriales a comprender el panorama de los libros mejor valorados en diferentes géneros. Tus conocimientos ayudarán a dar forma a futuras estrategias de marketing de libros y decisiones editoriales.

**Antecedentes**

En un mundo donde los datos se han convertido en la nueva moneda, las empresas están aprovechando los macrodatos para tomar decisiones informadas que impulsen el éxito y la rentabilidad. La industria editorial, al igual que otras, utiliza el análisis de datos para comprender las tendencias del mercado, las preferencias de los lectores y el rendimiento de los libros según factores como el género, el autor y las calificaciones. Books to Scrape sirve como una rica fuente de estos datos, ofreciendo información detallada sobre una amplia gama de libros, lo que lo convierte en una plataforma ideal para extraer conocimientos que ayuden a tomar decisiones informadas en el mundo literario.

**Tarea**

Tu tarea es crear un script en Python utilizando BeautifulSoup y pandas para extraer datos de libros de Books to Scrape, centrándote en las calificaciones y géneros de los libros. El script debe ser capaz de filtrar libros con calificaciones por encima de un umbral específico y en géneros específicos. Además, el script debe estructurar los datos extraídos en un formato tabular utilizando pandas para su posterior análisis.

**Resultado esperado**

Una función llamada `scrape_books` que tome dos parámetros: `min_rating` y `max_price`. La función debe extraer datos de libros del sitio web "Books to Scrape" y devolver un DataFrame de `pandas` con las siguientes columnas:

- **UPC**: El código universal del producto (UPC) del libro.
- **Título**: El título del libro.
- **Precio (£)**: El precio del libro en libras.
- **Calificación**: La calificación del libro (1-5 estrellas).
- **Género**: El género del libro.
- **Disponibilidad**: Si el libro está en stock o no.
- **Descripción**: Una breve descripción o descripción del producto del libro (si está disponible).

Ejecutarás este script para extraer datos de libros con una calificación mínima de `4.0 o superior` y un precio máximo de `£20`.

Recuerda experimentar con diferentes calificaciones y precios para asegurarte de que tu código sea versátil y pueda manejar diversas búsquedas de manera efectiva.

**Recursos**

- [Documentación de Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Documentación de Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Pista**

Tu primera misión es familiarizarte con el sitio web **Books to Scrape**. Navega a [Books to Scrape](http://books.toscrape.com/) y explora los libros disponibles para comprender su diseño y estructura.

A continuación, piensa en cómo puedes establecer parámetros para tu extracción de datos:

- **Calificación Mínima**: Concéntrate en libros con una calificación de 4.0 o superior.
- **Precio Máximo**: Filtra libros con un precio de hasta £20.

Después de revisar el sitio, puedes construir un plan para extraer los datos relevantes. Presta atención a los detalles mostrados para cada libro, incluyendo el título, precio, calificación y disponibilidad. Esto te ayudará a identificar los elementos HTML correctos a los que apuntar con tu script de scraping.

Asegúrate de construir tu URL y lógica de scraping basándote en los patrones que observes en la estructura HTML de las listas de libros.


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Nota Importante**:

En el mundo en constante cambio de internet, los sitios web a menudo actualizan y modifican sus estructuras. Cuando intentes este laboratorio, es posible que el sitio web **Books to Scrape** sea diferente de lo que esperas.

Si encuentras problemas debido a estos cambios, como nuevas reglas u obstáculos que impidan la extracción de datos, ¡no te preocupes! Sé creativo.

Puedes elegir otro sitio web que te interese y sea adecuado para extraer datos. Opciones como Wikipedia, The New York Times o incluso bases de datos de bibliotecas son excelentes alternativas. El objetivo principal sigue siendo el mismo: extraer datos útiles y mejorar tus habilidades de web scraping mientras exploras una fuente de información que disfrutes. ¡Esta es tu oportunidad para practicar y adaptarte a diferentes entornos web!

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

def scrape_books(min_rating=4.0, max_price=20.0):
    base_url = "http://books.toscrape.com/catalogue/"
    books = []

    # Función para convertir calificaciones de texto a números
    def rating_to_number(rating_text):
        ratings = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
        return ratings.get(rating_text, 0)

    # Iterar sobre las páginas del sitio
    page_url = "http://books.toscrape.com/catalogue/page-1.html"
    while page_url:
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extraer información de cada libro
        for book in soup.find_all("article", class_="product_pod"):
            title = book.h3.a["title"]
            price = float(book.find("p", class_="price_color").text[1:])
            rating = rating_to_number(book.p["class"][1])
            availability = book.find("p", class_="instock availability").text.strip()

            # Filtrar por calificación mínima y precio máximo
            if rating >= min_rating and price <= max_price:
                book_url = base_url + book.h3.a["href"]
                book_response = requests.get(book_url)
                book_soup = BeautifulSoup(book_response.content, "html.parser")

                # Extraer detalles adicionales
                upc = book_soup.find("th", text="UPC").find_next_sibling("td").text
                genre = book_soup.find("ul", class_="breadcrumb").find_all("li")[2].text.strip()
                description_tag = book_soup.find("meta", {"name": "description"})
                description = description_tag["content"].strip() if description_tag else "No description"

                books.append({
                    "UPC": upc,
                    "Título": title,
                    "Precio (£)": price,
                    "Calificación": rating,
                    "Género": genre,
                    "Disponibilidad": availability,
                    "Descripción": description
                })

        # Verificar si hay una página siguiente
        next_page = soup.find("li", class_="next")
        if next_page:
            next_page_url = next_page.a["href"]
            page_url = urllib.parse.urljoin(page_url, next_page_url)
        else:
            page_url = None


    # Crear un DataFrame con los datos extraídos
    return pd.DataFrame(books)

# Llamar a la función con los parámetros requeridos
books_df = scrape_books(min_rating=4.0, max_price=20.0)
print(books_df)

C:\Users\sergi\AppData\Local\Temp\ipykernel_19860\4096829912.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = book_soup.find("th", text="UPC").find_next_sibling("td").text


                 UPC                                             Título  \
0   ce6396b0f23f6ecc                                        Set Me Free   
1   6258a1f6a6dcfe50  The Four Agreements: A Practical Guide to Pers...   
2   6be3beb0793a53e7                                     Sophie's World   
3   657fe5ead67a7767            Untitled Collection: Sabbath Poems 2014   
4   51653ef291ab7ddc                                    This One Summer   
..               ...                                                ...   
70  9c96cd1329fbd82d                                    The Zombie Room   
71  b78deb463531d078                                    The Silent Wife   
72  4280ac3eab57aa5d                                  The Girl You Lost   
73  29fc016c459aeb14              The Edge of Reason (Bridget Jones #2)   
74  19fec36a1dfb4c16  A Spy's Devotion (The Regency Spies of London #1)   

    Precio (£)  Calificación              Género Disponibilidad  \
0        17.46             5    